In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.2. Correlation Analyses/1.2.1. Compartment Correlations/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.2. Correlation Analyses/1.2.1. Compartment Correlations/Output')
cur_date = '052021'

library(readxl)
library(tidyverse)
library(reshape2)

#reading in file
data_df <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(data_df)

,Original_Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,E_C_F_002,IFNg,17.642316,4.220509,NELF,1,NS,NS_1
2,E_C_F_002,IL10,2.873724,1.953721,NELF,1,NS,NS_1
3,E_C_F_002,IL12p70,1.625272,1.392467,NELF,1,NS,NS_1
4,E_C_F_002,IL13,36.117692,5.214035,NELF,1,NS,NS_1
5,E_C_F_002,IL1b,104.409217,6.719857,NELF,1,NS,NS_1
6,E_C_F_002,IL6,21.159536,4.469856,NELF,1,NS,NS_1


Running Spearman correlations tests to determine amount of correlations between compartments within ecig or cigarette smokers.

In [3]:
compartment = c('NLF','NELF','NLF','NLF','Sputum', 'NELF', # compartments for first df
                'NELF', 'Sputum', 'Sputum', 'Serum', 'Serum', 'Serum') #compartments for 2nd df
smokers = c('NS','CS', 'Ecig')

empty_vector = c()
for (i in 1:length(compartment)){
    for (j in 1:length(smokers)){
        if (i < 7){
            compartment1_df = data_df %>%
                filter(Compartment == compartment[i], Group == smokers[j]) %>%
                group_by(Protein) %>%
                summarize(Protein_Avg_Conc_pslog2 = mean(Conc_pslog2))
            compartment2_df = data_df %>%
                filter(Compartment == compartment[i + 6], Group == smokers[j]) %>%
                group_by(Protein) %>%
                summarize(Protein_Avg_Conc_pslog2 = mean(Conc_pslog2))
            spearman_test = cor.test(compartment1_df$Protein_Avg_Conc_pslog2, compartment2_df$Protein_Avg_Conc_pslog2, 
                                    method = c('spearman'))

            #remembering values of interest
            values_vector = c(compartment[i], compartment[i + 6], smokers[j], spearman_test$estimate, spearman_test$p.value)
            empty_vector = c(empty_vector, values_vector)
        }
    }
}

In [4]:
#putting everything into a table
dim(empty_vector) = c(5, 18)
table = data.frame(t(empty_vector))
colnames(table) = c('Compartment 1', 'Compartment 2', 'Smoking Status', 'Spearman Estimate', 'P Value')
final_table = data.frame(table %>%
                        arrange(`Smoking Status`))
final_table

Compartment.1,Compartment.2,Smoking.Status,Spearman.Estimate,P.Value
<fct>,<fct>,<fct>,<fct>,<fct>
NLF,NELF,CS,0.958215697346132,3.77262311925849e-06
NELF,Sputum,CS,0.916431394692264,3.98548399071368e-06
NLF,Sputum,CS,0.897233201581028,3.40029052874982e-06
NLF,Serum,CS,0.638622247317899,0.00176356741999373
Sputum,Serum,CS,0.656691134952004,0.00119158427287967
NELF,Serum,CS,0.731225296442688,0.000167869737836107
NLF,NELF,Ecig,0.970638057594579,3.43483964594474e-06
NELF,Sputum,Ecig,0.917560700169396,4.00520419443953e-06
NLF,Sputum,Ecig,0.872388481084133,2.07253971041151e-06


In [5]:
#exporting data
write.csv(final_table, paste0(Output,"/", cur_date, "_Spearman_Correlations.csv"), row.names = FALSE)